# __Basic Machiene Learning project__
## Kaggle Titanic Dataset 

#### _In this project which passenger survived is predicted after titanic disaster by using logistic regression._

#### Importing the relevant packages. 

In [38]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression

#### Loading the train data and displaying.

In [39]:
raw_data = pd.read_csv("./data/train.csv")
#pd.set_option("display.max_rows", None)
raw_data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# __Preprocessing__

#### _In this code block the cabin data of the passengers is processed by using some string operations and created new data frame. The new data frame contains columns for letters from cabin code and the number inside the code._  

In [40]:
cabin_df = pd.DataFrame({
    "letter_A": np.zeros(len(raw_data)),
    "letter_B": np.zeros(len(raw_data)),
    "letter_C": np.zeros(len(raw_data)),
    "letter_D": np.zeros(len(raw_data)),
    "letter_E": np.zeros(len(raw_data)),
    "letter_F": np.zeros(len(raw_data)),
    "number": np.zeros(len(raw_data)),
})

n_count = 0
count = 0
for i in range(len(raw_data)):
    cbn_str = raw_data["Cabin"][i]
    if  type(cbn_str) != float:
        for letter in ["A", "B", "C", "D", "E", "F"]:
            if cbn_str.count(letter) > 0:
                cabin_df.loc[i]["letter_"+letter] = 1
        nmbr = int("0"+cbn_str.split(" ")[0][1:])
        n_count += nmbr 
        count += 1
        cabin_df.loc[i]["number"] = nmbr
        
mean_number = n_count / count
for i in range(len(raw_data)):
    if cabin_df["number"][i] == 0:
        cabin_df.loc[i]["number"] = mean_number


cabin_df

,letter_A,letter_B,letter_C,letter_D,letter_E,letter_F,number
0,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
1,0.0,0.0,1.0,0.0,0.0,0.0,85.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
3,0.0,0.0,1.0,0.0,0.0,0.0,123.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
...,...,...,...,...,...,...,...
886,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
887,0.0,1.0,0.0,0.0,0.0,0.0,42.000000
888,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
889,0.0,0.0,1.0,0.0,0.0,0.0,148.000000


#### _There are some nan values in the Age column. This code block replaces the nan values by the average age value of all passengers._

In [41]:
mean = raw_data["Age"].mean()
age = pd.DataFrame(raw_data["Age"])
for col in range(len(age["Age"])):
    if np.isnan(age["Age"][col]):
        age.loc[col] = mean  

#### We can see that there are no nan values in the new age column.

In [42]:
age["Age"].isnull().values.sum()

0

#### _In this code block the dummy veriables are created and combining the all preprocessed data in to one dataframe named preprop_data._

In [43]:
dummies_Pclass = pd.get_dummies(raw_data["Pclass"])
dummies_sex = pd.get_dummies(raw_data["Sex"])
dummies_embarked = pd.get_dummies(raw_data["Embarked"])

dummies_Pclass =  dummies_Pclass.rename(columns = {1:"Pclass_1", 2:"Pclass_2", 3:"Pclass_3"})

preprop_data = pd.concat([age["Age"],
                          raw_data["SibSp"],
                          raw_data["Parch"],
                          raw_data["Fare"],
                          dummies_Pclass,
                          dummies_sex,
                          dummies_embarked,
                          cabin_df], axis=1)

preprop_data

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,female,male,C,Q,S,letter_A,letter_B,letter_C,letter_D,letter_E,letter_F,number
0,22.000000,1,0,7.2500,False,False,True,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
1,38.000000,1,0,71.2833,True,False,False,True,False,True,False,False,0.0,0.0,1.0,0.0,0.0,0.0,85.000000
2,26.000000,0,0,7.9250,False,False,True,True,False,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
3,35.000000,1,0,53.1000,True,False,False,True,False,False,False,True,0.0,0.0,1.0,0.0,0.0,0.0,123.000000
4,35.000000,0,0,8.0500,False,False,True,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.000000,0,0,13.0000,False,True,False,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
887,19.000000,0,0,30.0000,True,False,False,True,False,False,False,True,0.0,1.0,0.0,0.0,0.0,0.0,42.000000
888,29.699118,1,2,23.4500,False,False,True,True,False,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,48.137255
889,26.000000,0,0,30.0000,True,False,False,False,True,True,False,False,0.0,0.0,1.0,0.0,0.0,0.0,148.000000


## Normalization.

In [44]:
target = raw_data["Survived"].to_numpy()
scalar = preprocessing.MinMaxScaler()
#new = preprop_data.to_numpy()
new = scalar.fit_transform(preprop_data)

## Creating and fitting the model.

In [45]:
reg = LogisticRegression()
reg.fit(new, target)

LogisticRegression()

### We can see the precision of the model is about %80.

In [46]:
reg.score(new,target)

0.8125701459034792

## Loading the test data.

In [47]:
test = pd.read_csv("./data/test.csv")
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


#### _In this code block the same preprocessing operations are applied to the test dataset._

In [48]:
cabin_df_test = pd.DataFrame({
    "letter_A": np.zeros(len(test)),
    "letter_B": np.zeros(len(test)),
    "letter_C": np.zeros(len(test)),
    "letter_D": np.zeros(len(test)),
    "letter_E": np.zeros(len(test)),
    "letter_F": np.zeros(len(test)),
    "number": np.zeros(len(test)),
})

n_count_t = 0
count_t = 0
for i in range(len(test)):
    cbn_str = test["Cabin"][i]
    if  type(cbn_str) != float:
        for letter in ["A", "B", "C", "D", "E", "F"]:
            if cbn_str.count(letter) > 0:
                cabin_df_test.loc[i]["letter_"+letter] = 1
        nmbr = int("0"+cbn_str.split(" ")[0][1:])
        n_count_t += nmbr 
        count_t += 1
        cabin_df_test.loc[i]["number"] = nmbr
        
mean_number = n_count_t / count_t
for i in range(len(test)):
    if cabin_df_test["number"][i] == 0:
        cabin_df_test.loc[i]["number"] = mean_number


mean = test["Age"].mean()
mean_fare = test["Fare"].mean()
age = pd.DataFrame(test["Age"])
for col in range(len(age["Age"])):
    if np.isnan(age["Age"][col]):
        age.loc[col] = mean
for col in range(len(test)):
    if np.isnan(test["Fare"][col]):
        test.loc[col, "Fare"] = mean_fare
        
dummies_Pclass = pd.get_dummies(test["Pclass"])
dummies_sex = pd.get_dummies(test["Sex"])
dummies_embarked = pd.get_dummies(test["Embarked"])

dummies_Pclass =  dummies_Pclass.rename(columns = {1:"Pclass_1", 2:"Pclass_2", 3:"Pclass_3"})

preprop_test = pd.concat([age["Age"],
                          test["SibSp"],
                          test["Parch"],
                          test["Fare"],
                          dummies_Pclass,
                          dummies_sex,
                          dummies_embarked,
                          cabin_df_test], axis=1)



test_final = scalar.fit_transform(preprop_test)
preprop_test

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,female,male,C,Q,S,letter_A,letter_B,letter_C,letter_D,letter_E,letter_F,number
0,34.50000,0,0,7.8292,False,False,True,False,True,False,True,False,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
1,47.00000,1,0,7.0000,False,False,True,True,False,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
2,62.00000,0,0,9.6875,False,True,False,False,True,False,True,False,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
3,27.00000,0,0,8.6625,False,False,True,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
4,22.00000,1,1,12.2875,False,False,True,True,False,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,30.27259,0,0,8.0500,False,False,True,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
414,39.00000,0,0,108.9000,True,False,False,True,False,True,False,False,0.0,0.0,1.0,0.0,0.0,0.0,105.00000
415,38.50000,0,0,7.2500,False,False,True,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,44.78022
416,30.27259,0,0,8.0500,False,False,True,False,True,False,False,True,0.0,0.0,0.0,0.0,0.0,0.0,44.78022


### Evaluating the results by using predict method.

In [49]:
result = reg.predict(test_final)
result

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

#### Finally, creating the dataframe for kaggle submission and saving it as csv file. This model gets __0.767__ score for Titanic competitions. 

In [53]:
submission = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": result})
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [54]:
submission.to_csv("titanic_submission.csv", index=False)